# Cloud Pak for Data Dashboard
This notebook will connect to a CP4D cluster and publish the information to a Dashboard.

In [ ]:
%run db2.ipynb
%run connection.ipynb

### Add Dashboard Extensions

In [ ]:
%run dashboard.ipynb

The dashboard notebook contains a number of functions that provide RESTful services to a Dashboard service. This dashboard service runs on port 7272 and communicates through RESTful API calls. The functions that are provided in the dashboard notebook includes:

**Publishing functions**
- publishPie - cell, title, data
- publishBar - cell, title, description, ticks, data
- publishLine - cell, title, description, data [,trend=False]
- publishText - cell, title, description, bigvalue, (label1/value1 ... label4/value4)
- publishListing - cell, [labels], [text], [description] (up to 9 items)

**Cells Available for Updating**
- stock_info - Text
- stock_ytd - Line
- stock_purchase - Listing
- top_stocks - Pie
- stock_30day - Line

The Dashboard can run locally on this image (port 7272) or it can run against a remote server. The next cell is set to have the Dashboard run locally, so you must make sure that the service has been properly started (see the notebook on Portainer and starting the dashboard service). The other URL that is provided here is for an AWS server that has been set up to provide this Dashboard service. If you want to use that one then uncomment that line.

In [ ]:
seturl('http://localhost:7272')

### Select a Stock Symbol from the following list
Select one main stock and two others to compare against that are currently available from the following list.

In [ ]:
%%sql -grid
WITH SYMBOLS(SYMBOL,NAME) AS (
  SELECT TRIM(SYMBOL), TRIM(COMPANY) FROM DVDEMO.STOCK_SYMBOLS
)
SELECT H.SYMBOL, S.NAME, COUNT(*) FROM FOLDING.STOCK_HISTORY H, SYMBOLS S
WHERE H.SYMBOL = S.SYMBOL
GROUP BY H.SYMBOL, S.NAME

Before updating the Dashboard, click on the following link to open up a separate browser window.

[Dashboard](http://localhost:7272)

The following code will generate the calls necessary to publish information to the Dashboard service.

In [ ]:
def publish_dashboard(stock1_in, stock2_in, stock3_in, stock_name_in):
    
    stock1 = stock1_in
    stock2 = stock2_in
    stock3 = stock3_in
    weeks = [25, 45]
    stock_name = stock_name_in
    
    # Step 2: Get the open price of the stock and the volume for the first record found
    stock_values = %sql SELECT OPEN, CLOSE, VOLUME FROM FOLDING.STOCK_HISTORY \
                        WHERE SYMBOL = :stock1 FETCH FIRST ROW ONLY
    stock_open = str(stock_values['OPEN'][0])
    stock_current = str(stock_values['CLOSE'][0])
    stock_volume = int(stock_values['VOLUME'][0])

    # Step 3: Publish the information into cell STOCK_INFO
    publishText('stock_info',stock1,stock_name,stock_current,'Open',stock_open,'Volume',stock_volume)

    # Step 4: Get the YTD stock values on a weekly basis
    sql = '''
    WITH WEEKLY(SYMBOL, WEEKNO, VALUE) AS (
       SELECT SYMBOL, WEEK(TX_DATE), DEC(AVG(OPEN),9,2)
         FROM FOLDING.STOCK_HISTORY
         WHERE SYMBOL IN (:stock1, :stock2, :stock3) 
               AND TX_DATE != '2017-12-01' 
               AND WEEK(TX_DATE) BETWEEN :weeks[0] AND :weeks[1]
         GROUP BY SYMBOL, WEEK(TX_DATE)
    )
    SELECT * FROM WEEKLY 
    ORDER BY SYMBOL, WEEKNO
    '''
    data = %sql -array {sql}

    stock1_volume = []
    stock2_volume = []
    stock3_volume = []
    for row in data[1:]:
        if (row[0] == stock1): stock1_volume.append([row[1],row[2]])
        if (row[0] == stock2): stock2_volume.append([row[1],row[2]])
        if (row[0] == stock3): stock3_volume.append([row[1],row[2]])


    # Step 5: Publish the data into the cell STOCK_YTD
    publishLine('stock_ytd','{0} vs {1} vs {2}'.format(stock1,stock2,stock3),'Stock Price Average Last 20 Weeks', 
                [stock1,stock2,stock3],[stock1_volume, stock2_volume, stock3_volume])

    # Step 6: Top Purchase/Sell of Stock today 
    
    sql = '''
    WITH
    TX_DAY(TX_DATE) AS (
      VALUES NEXT_DAY(DATE('2018-01-01') + INT(RAND()*350) DAYS, 'Thursday')
    ),
    PURCHASED(CUSTID,AMOUNT) AS (
      SELECT CUSTID, SUM(QUANTITY) AS AMOUNT
        FROM FOLDING.STOCK_TRANSACTIONS_DV ST, TX_DAY TX
      WHERE ST.TX_DATE = TX.TX_DATE 
      GROUP BY CUSTID
    )
    SELECT C.LASTNAME, C.CITY, C.STATE, P.AMOUNT
      FROM PURCHASED P, TRADING.CUSTOMERS C
      WHERE C.CUSTID = P.CUSTID
    ORDER BY ABS(P.AMOUNT) DESC
    FETCH FIRST 9 ROWS ONLY 
    '''
    data = %sql -array {sql}
    customer = []
    purchase = []
    location = []
    color    = []
    
    row_count = 0
    for row in data[1:]:
        row_count = row_count + 1
        customer.append(row[0])
        
        if (int(row[3]) < 0):
            color.append("red")
        else:
            color.append("green")
            
        purchase.append(row[3])
        location.append("{0}, {1}".format(row[1],row[2]))
    
    if (row_count == 0): # No data found
        customer.append("No stock trades completed")
        color.append("blue")
        location.append("")
    
    # Step 7: Publish the result to the cell STOCK_COMPARE    
    publishListing('stock_purchase',purchase,color,customer,location)

    # Step 8: Compare the stocks by capitalization

    sql = '''
    WITH WEEK45(SYMBOL, PURCHASES) AS (
      SELECT SYMBOL, SUM(VOLUME * CLOSE) FROM FOLDING.STOCK_HISTORY
        WHERE WEEK(TX_DATE) =  45 AND SYMBOL IN (:stock1, :stock2, :stock3)
      GROUP BY SYMBOL
    )
    SELECT SYMBOL, PURCHASES FROM WEEK45
    ORDER BY SYMBOL, PURCHASES
    '''

    data = %sql -array {sql}

    # Step 9: Publish the result to the cell TOP_STOCKS   

    publishPie('top_stocks','Stock Capitalization',data[1:])

    # Step 10: 30 Day Moving Average with trend line

    sql = '''
    WITH MOVAVG(WEEKNO, OPEN, MOVING) AS (
       SELECT WEEK(TX_DATE) AS WEEK, OPEN, 
         AVG(OPEN) OVER (
           ORDER BY TX_DATE
         ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING) AS MOVING_AVG
      FROM FOLDING.STOCK_HISTORY
         WHERE SYMBOL = :stock1
      ORDER BY WEEK(TX_DATE)
    )
    SELECT WEEKNO, DEC(AVG(OPEN),9,2), DEC(AVG(MOVING),9,2) FROM MOVAVG
      WHERE WEEKNO BETWEEN 10 AND 20
    GROUP BY WEEKNO
    ORDER BY WEEKNO
    '''
    data = %sql -array {sql}

    stock1_volume = []
    stock2_volume = []
    for row in data[1:]:
        stock1_volume.append([row[0],row[1]])
        stock2_volume.append([row[0],row[2]])
    
    # Step 11: Publish the trend line to STOCK_30DAY
    
    publishLine('stock_30day',stock1,'30-day Moving Average',['Base','Trend'],[stock1_volume,stock2_volume])
    print("Published to Dashboard")

Select a group of Stock symbols to compare.

In [ ]:
stock1 = 'INTC'
stock2 = 'AAPL'
stock3 = 'MSFT'
weeks = [ 25, 45 ]

Click on the following link to publish the data to the Dashboard service.

In [ ]:
publish_dashboard(stock1, stock2, stock3, "Intel Corporation")